In [11]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from pprint import pprint
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
from tensorflow import keras
import tensorflow as tf
import warnings
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import cross_val_score

from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectPercentile
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
plt.rc('font', family=fm.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()) # for Windows OS user
import datetime
import os

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import BayesianRidge
from tensorflow import keras
from scipy.stats.mstats import gmean
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")
    

In [13]:
# 학습용과 평가용(제출용) 데이터를 읽어들인다.
trn = pd.read_csv('X_train.csv', encoding='cp949')
tst = pd.read_csv('X_test.csv', encoding='cp949')

# 학습용 정답 데이터를 읽는다.
y = pd.read_csv('y_train.csv', encoding='cp949')

In [14]:
IDtrain = trn.custid.unique()
IDtest = tst.custid.unique()
data = pd.concat([trn, tst], axis=0)

In [15]:
IDtest

array([30001, 30002, 30003, ..., 49992, 49993, 49994], dtype=int64)

날짜 데이터

In [16]:
data['sales_year']=data['sales_month'].apply(lambda x: 2018 if x >12 else 2017)

In [17]:
# 월 정리
data['sales_month']=data['sales_month'].apply(lambda x: x-12 if x >12 else x)

In [18]:
# 주말
data['weekend'] = data['sales_dayofweek'].apply(lambda x: 1 if (x=='토' or x=='일') else 0)

In [19]:
data['sales_year']=data['sales_year'].astype('str')
data['sales_month']=data['sales_month'].astype('str')
data['sales_day']=data['sales_day'].astype('str')

In [20]:
data['date_time']=data['sales_year']+'-'+data['sales_month']+'-'+data['sales_day']

In [21]:
data['sales_year']=data['sales_year'].astype('int')
data['sales_month']=data['sales_month'].astype('int')
data['sales_day']=data['sales_day'].astype('int')

In [22]:
data['date_time'] = pd.to_datetime(data['date_time'])

In [23]:
a=data.groupby(['custid'])['date_time'].diff()

In [24]:
a=a.reset_index().set_index(data.custid)

In [25]:
a=a.reset_index()

In [26]:
a.set_index('index',inplace=True)

In [27]:
data['date_time_diff']=a.date_time

In [28]:
dtdt=data.groupby(['custid','date_time'])['date_time_diff'].sum().reset_index()

In [29]:
dtdt.date_time_diff=dtdt.date_time_diff.astype('str')

In [30]:
dtdt.date_time_diff=dtdt.date_time_diff.apply(lambda x: x.split(' ')[0])

In [31]:
dtdt.date_time_diff=dtdt.date_time_diff.astype('int')

In [32]:
dtdt_mean=dtdt.groupby('custid')['date_time_diff'].mean().reset_index()

# 쇼핑시간

In [33]:
dtdt_mean

,custid,date_time_diff
0,0,41.142857
1,2,18.428571
2,3,25.461538
3,4,1.500000
4,5,15.333333
...,...,...
35962,49988,28.000000
35963,49990,0.000000
35964,49992,142.000000
35965,49993,0.000000


In [34]:
ab=data.groupby(['custid','sales_day','sales_month'])['sales_time'].agg([('sales_time_min', 'min')]).reset_index()

In [35]:
abc=data.groupby(['custid','sales_day','sales_month'])['sales_time'].agg([('sales_time_max', 'max')]).reset_index()

In [36]:
ab['time_min']=ab['sales_time_min'].apply(lambda x: int(str(x)[:2]) if len(str(x)) == 4 else int(str(x)[:1]))
ab['minute_min']=ab['sales_time_min'].apply(lambda x: int(str(x)[2:]) if len(str(x)) == 4 else int(str(x)[1:]))

In [37]:
ab['time_max']=abc['sales_time_max'].apply(lambda x: int(str(x)[:2]) if len(str(x)) == 4 else int(str(x)[:1]))
ab['minute_max']=abc['sales_time_max'].apply(lambda x: int(str(x)[2:]) if len(str(x)) == 4 else int(str(x)[1:]))

In [38]:
ab['shop_time']=(ab['time_max']-ab['time_min'])*60+(ab['minute_max']-ab['minute_min'])

In [39]:
shop_time=ab.groupby('custid')['shop_time'].mean().reset_index()

### word2vec

In [40]:
# income_occyp_type, income_age_type, edu_occyp_type, edu_income_type, family_income_type

import random

trn2=trn.reset_index().drop('index',axis=1).copy()
trn2=trn2.reset_index()
trn2['goodcd']=trn2['goodcd'].astype('str')
train_data=list(trn2.groupby('custid')['goodcd'].unique())

In [41]:
def oversample(x, n):
    lst = []
    for i in x:
        tmp = []
        for j in range(n):
            random.shuffle(i)
            tmp += list(i)
            lst.append(tmp)
    return lst

In [42]:
w2v_input = oversample(train_data, 5)

In [43]:
from gensim.models import word2vec

w2v = word2vec.Word2Vec(sentences = w2v_input, vector_size = 100, window = 3, min_count = 1, sg = 1)

In [44]:
train_mean_vector = []
for words in tqdm(train_data):
    tmp = np.zeros(100)
    cnt = 0
    for word in words:
        try:
            tmp += w2v.wv[word]
            cnt += 1
        except:
            pass
    tmp /= cnt
    train_mean_vector.append(tmp)
train_mean_vector = np.array(train_mean_vector)

100%|█████████████████████████████████████████████████████████████████████████| 21587/21587 [00:01<00:00, 15635.28it/s]


In [45]:
train_mean_vector = pd.DataFrame(train_mean_vector)
train_mean_vector

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.061542,0.056873,0.088633,-0.110187,0.010319,-0.077438,0.204967,0.183569,0.014509,-0.092617,...,0.070798,0.025625,0.058644,-0.255308,-0.078397,-0.032968,0.098925,-0.103489,0.137099,-0.204718
1,0.016118,0.218577,-0.020153,0.187573,0.152344,-0.182511,-0.095017,0.268245,0.077921,-0.162361,...,0.089718,0.017023,0.004338,-0.448536,-0.009264,-0.198337,0.127382,-0.066545,0.031457,0.138095
2,-0.255369,0.218157,-0.062035,0.139190,0.045286,0.021811,-0.012492,0.294170,0.007780,0.081570,...,0.112953,-0.037676,-0.002447,-0.299069,-0.098315,0.108659,0.083755,-0.067097,0.213432,0.078868
3,-0.146511,-0.182469,-0.042544,0.019734,0.164819,-0.295148,0.095127,0.102150,-0.174055,-0.172746,...,0.642603,-0.225270,-0.217359,-0.331290,0.005355,0.241408,0.404333,-0.541870,-0.025808,-0.193828
4,-0.253609,0.185963,-0.004582,0.016257,-0.073511,0.002255,0.106225,0.225060,-0.091049,-0.102035,...,0.120553,0.054224,-0.115468,-0.143754,-0.183742,0.004837,0.081039,-0.017392,0.130946,-0.160380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21582,0.038952,0.158030,-0.018494,0.029227,0.110629,-0.085440,0.031798,0.331952,-0.061453,-0.094627,...,0.144935,0.105126,0.078305,-0.174782,0.005704,0.007209,0.105265,-0.244889,0.199799,-0.219074
21583,-0.006507,0.136995,-0.149236,0.002044,0.153614,-0.044837,0.153875,0.296153,-0.047018,-0.185537,...,0.062877,0.037913,0.050434,-0.154289,0.093573,-0.018004,0.282097,0.026103,0.176460,0.068016
21584,-0.170812,0.180552,0.038717,0.137382,-0.135571,-0.107863,0.021959,0.352930,0.044536,-0.074014,...,0.081747,0.177766,0.186964,-0.216684,-0.105557,0.200159,0.293047,-0.049944,0.018963,-0.077992
21585,-0.092500,0.175067,-0.063254,-0.092458,-0.008215,-0.052605,0.119415,0.315698,0.046452,-0.107578,...,0.194209,-0.006328,-0.052689,-0.088868,0.096413,0.091933,0.323266,-0.196742,0.170948,0.097792


In [46]:
# income_occyp_type, income_age_type, edu_occyp_type, edu_income_type, family_income_type

import random

tst2=tst.reset_index().drop('index',axis=1).copy()
tst2=tst2.reset_index()
tst2['goodcd']=tst2['goodcd'].astype('str')
test_data=list(tst2.groupby('custid')['goodcd'].unique())

In [47]:
test_mean_vector = []
for words in tqdm(test_data):
    tmp = np.zeros(100)
    cnt = 0
    for word in words:
        try:
            tmp += w2v.wv[word]
            cnt += 1
        except:
            pass
    tmp /= cnt
    test_mean_vector.append(tmp)
test_mean_vector = np.array(test_mean_vector)

100%|█████████████████████████████████████████████████████████████████████████| 14380/14380 [00:00<00:00, 15793.63it/s]


In [48]:
test_mean_vector = pd.DataFrame(test_mean_vector)
test_mean_vector

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.132930,0.155734,0.151646,0.068456,-0.003697,0.029520,0.062507,0.306939,-0.029330,-0.108594,...,0.094814,0.101818,-0.091067,-0.241184,-0.226995,-0.113442,0.063421,0.013044,0.245279,-0.126025
1,-0.088401,0.128908,-0.018629,0.100338,-0.011866,-0.049685,0.047318,0.276287,-0.005998,-0.084904,...,0.144259,0.092630,-0.031617,-0.163453,-0.072596,0.079954,0.152019,-0.090272,0.199531,-0.066211
2,-0.267335,0.228172,0.019315,-0.132455,-0.063226,0.364643,0.143861,0.386225,-0.090614,-0.156111,...,0.342110,0.007563,-0.049062,-0.197961,0.085084,-0.012924,0.153711,0.174028,0.130789,0.029266
3,-0.019369,0.204129,0.041491,0.009992,-0.136156,-0.041238,0.185946,0.397358,0.007071,-0.109215,...,0.051666,-0.045467,0.057830,-0.152573,0.187562,-0.174977,0.091271,-0.044189,0.130247,-0.099189
4,-0.315853,0.271953,-0.079658,-0.128518,-0.017075,0.165995,0.259495,0.355456,0.113665,-0.324010,...,0.217618,-0.102182,0.079764,-0.071714,0.030500,0.010211,0.135265,0.039706,0.010439,-0.083185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14375,-0.130912,0.259859,0.064655,-0.042632,-0.106299,0.156177,-0.074448,0.284330,-0.098886,-0.292375,...,0.204193,0.321694,-0.156073,-0.047712,-0.197576,0.056773,-0.133331,-0.099340,0.150410,-0.081173
14376,-0.267657,0.182745,-0.002856,0.107635,-0.025713,-0.139545,0.266818,0.387936,0.036856,-0.273038,...,0.079278,0.009070,0.081375,-0.105646,0.007727,-0.100705,0.101752,0.094941,0.066065,-0.021121
14377,-0.025629,0.126905,0.038476,-0.160173,0.031174,-0.297837,-0.084994,0.143794,-0.142578,-0.042474,...,0.157538,0.065261,-0.075927,-0.296062,-0.267985,0.077667,0.160068,0.034527,0.141176,-0.216591
14378,-0.661867,0.023487,0.162321,-0.021873,-0.142295,0.216998,0.022104,0.288725,0.051792,-0.222829,...,0.198817,0.047374,-0.052840,-0.155514,-0.032686,0.121621,0.046995,0.130970,0.102449,-0.044051


In [49]:
# income_occyp_type, income_age_type, edu_occyp_type, edu_income_type, family_income_type

import random

trn2=trn.reset_index().drop('index',axis=1).copy()
trn2=trn2.reset_index()
train_data=list(trn2.groupby('custid')['brd_nm'].unique())

In [50]:
def oversample(x, n):
    lst = []
    for i in x:
        tmp = []
        for j in range(n):
            random.shuffle(i)
            tmp += list(i)
            lst.append(tmp)
    return lst

In [51]:
w2v_input = oversample(train_data, 5)

In [52]:
from gensim.models import word2vec

w2v = word2vec.Word2Vec(sentences = w2v_input, vector_size = 100, window = 3, min_count = 1, sg = 1)

In [53]:
train_mean_vector_2 = []
for words in tqdm(train_data):
    tmp = np.zeros(100)
    cnt = 0
    for word in words:
        try:
            tmp += w2v.wv[word]
            cnt += 1
        except:
            pass
    tmp /= cnt
    train_mean_vector_2.append(tmp)
train_mean_vector_2 = np.array(train_mean_vector_2)

100%|█████████████████████████████████████████████████████████████████████████| 21587/21587 [00:01<00:00, 18009.12it/s]


In [54]:
train_mean_vector_2 = pd.DataFrame(train_mean_vector_2)
train_mean_vector_2

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.120581,-0.168126,-0.093358,0.130806,-0.115939,-0.310098,0.031862,-0.007990,-0.103392,-0.266050,...,-0.165226,0.226791,0.023077,0.197462,0.159934,0.134166,-0.084717,0.058027,0.233149,0.146704
1,0.016066,-0.175732,-0.060114,0.039251,-0.101751,-0.212600,-0.026284,-0.178640,-0.092970,-0.138123,...,-0.212440,0.137763,-0.042228,0.076557,0.097435,0.117673,-0.081022,0.041562,0.235905,0.100596
2,-0.123167,-0.187222,-0.063106,0.032984,-0.149452,-0.086457,-0.035880,-0.056020,-0.162613,-0.107509,...,-0.185209,0.162313,-0.024075,0.080504,0.102624,0.079979,-0.165694,0.105364,0.314384,0.118440
3,-0.193487,-0.087682,-0.101784,0.144094,-0.219583,-0.063452,-0.002532,-0.056295,-0.245091,-0.218480,...,-0.051455,0.020150,-0.073568,0.343153,0.258331,0.246874,-0.016653,0.158458,0.133428,0.105987
4,0.030284,-0.195019,-0.061709,0.004915,-0.098922,-0.128137,-0.058516,-0.095405,-0.190178,-0.147233,...,-0.179383,0.158299,-0.097633,0.092266,0.095512,0.130884,-0.039362,0.080870,0.225688,0.169993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21582,-0.018193,-0.183459,-0.021244,0.051140,-0.116421,-0.131396,0.043906,-0.058672,-0.237357,-0.259427,...,-0.087380,0.180822,-0.091415,0.093610,0.085881,0.134376,-0.136256,0.114941,0.086217,0.114949
21583,-0.011796,-0.231376,-0.018073,0.042466,-0.086801,-0.058015,-0.066308,-0.086597,-0.132022,-0.065993,...,-0.095967,0.147775,-0.108850,0.159752,0.103106,0.166150,-0.158565,0.209097,0.172259,0.031050
21584,0.090069,-0.142702,0.014944,-0.069143,-0.054553,-0.193201,0.050302,0.094423,-0.233922,-0.242887,...,-0.083197,0.235958,-0.067154,0.069837,0.179463,0.112109,-0.186623,0.223913,0.138100,0.103012
21585,-0.084705,-0.183849,-0.095647,0.003566,-0.171866,-0.028044,-0.035338,-0.066382,-0.156098,-0.093578,...,-0.041586,0.228185,-0.060137,0.210461,0.191180,0.128997,-0.136482,0.187943,0.338590,-0.023360


In [55]:
# income_occyp_type, income_age_type, edu_occyp_type, edu_income_type, family_income_type

import random

tst2=tst.reset_index().drop('index',axis=1).copy()
tst2=tst2.reset_index()
test_data=list(tst2.groupby('custid')['brd_nm'].unique())

In [56]:
test_mean_vector_2 = []
for words in tqdm(test_data):
    tmp = np.zeros(100)
    cnt = 0
    for word in words:
        try:
            tmp += w2v.wv[word]
            cnt += 1
        except:
            pass
    tmp /= cnt
    test_mean_vector_2.append(tmp)
test_mean_vector_2 = np.array(test_mean_vector_2)

100%|█████████████████████████████████████████████████████████████████████████| 14380/14380 [00:00<00:00, 17795.38it/s]


In [57]:
test_mean_vector_2 = pd.DataFrame(test_mean_vector_2)
test_mean_vector_2

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.017443,-0.174621,-0.105194,0.011899,0.015266,-0.155299,0.077563,-0.047686,-0.140162,-0.238857,...,-0.204700,0.160036,-0.023996,0.117331,0.129160,0.090782,-0.019356,0.105623,0.229560,0.123024
1,-0.001908,-0.132507,-0.050017,0.088914,-0.058232,-0.131739,0.002461,-0.048332,-0.209193,-0.121247,...,-0.122746,0.163274,-0.042743,0.061766,0.139213,0.074474,-0.141511,0.083415,0.215886,0.169451
2,0.077198,-0.252317,-0.003949,0.045440,-0.070676,-0.147768,-0.033464,-0.082746,-0.148074,-0.085137,...,-0.123839,0.167545,-0.121078,0.056365,0.037585,0.137053,-0.181564,0.106795,0.273853,0.106458
3,0.086740,-0.139128,-0.047973,0.097944,-0.192437,-0.067731,-0.034565,-0.023757,-0.231951,-0.124877,...,-0.046731,0.147687,-0.066205,0.127491,0.094139,0.247584,-0.109124,0.215378,0.163965,-0.084136
4,0.028153,-0.261091,0.026309,0.011200,-0.137154,-0.117949,-0.057408,-0.006317,-0.227226,-0.186583,...,-0.125063,0.093647,-0.038744,-0.011845,0.186116,0.169659,-0.120947,0.032807,0.301125,0.076763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14375,0.134864,-0.288073,-0.002128,0.104435,0.069870,-0.147911,0.061783,-0.278469,-0.124548,-0.146876,...,-0.129445,0.028837,-0.095100,-0.056888,0.088281,0.120404,-0.018461,0.112555,0.110769,0.399279
14376,0.050831,-0.238909,-0.061822,0.110594,-0.069637,-0.186823,-0.132783,0.046306,-0.215126,-0.028745,...,-0.170372,0.140821,-0.138006,0.058354,0.165498,0.221782,-0.135942,0.078361,0.424393,0.120720
14377,-0.128984,-0.219644,-0.163106,0.272883,-0.012682,-0.212286,-0.145577,-0.077054,-0.408790,-0.232035,...,-0.076609,0.184695,-0.107765,0.018057,0.328900,0.078679,0.170413,0.189459,0.134358,0.458461
14378,-0.018579,-0.217616,0.077768,-0.010903,-0.074095,-0.099657,0.018693,-0.167801,-0.219371,-0.117369,...,-0.186932,0.034624,0.025919,-0.010157,0.110671,0.070488,-0.110652,0.078641,0.344327,0.141736


In [58]:
# sales time - 아침/ 낮/ 밤 구매 나누기
data['sales_time'] = data['sales_time'].apply(lambda x: int(str(x)[:2]) if len(str(x)) == 4 else int(str(x)[:1]))
data['sales_time_cat'] = data['sales_time'].apply(lambda x: 'mo_sales' if (9 <= x <= 12) else ('af_sales' if (13 <= x <= 18) else 'knt_sales'))

In [59]:
# dis_amt - 할인율 특성 추가
data['dis_rate'] = data['dis_amt']/data['tot_amt']

In [60]:
# 퇴근전/후 구매 빈도, 비율
def get_off_work(x):
    if 9<=x<=17:
        return('bf_work')
    else:
        return('af_work')

In [61]:
data['sales_type'] = data['sales_time'].map(get_off_work)

시간별 구매 횟수

In [62]:
freq_per_time = pd.pivot_table(index='custid',columns='sales_time',values='tot_amt',aggfunc=np.size, fill_value=0, data=data.query("tot_amt>0")[['custid','sales_time','tot_amt']]).reset_index()

In [63]:
sales_type = pd.pivot_table(index='custid',columns='sales_type',values='tot_amt',aggfunc=np.size, fill_value=0,data = data.query('tot_amt>0')[['custid','sales_type','tot_amt']]).reset_index()

In [64]:
freq_per_time.columns=['custid', '1_stf', '9_stf', "10_stf", '11_stf', '12_stf', '13_stf', '14_stf', '15_stf', '16_stf', '17_stf', '18_stf', '19_stf', '20_stf', '21_stf', '22_stf']

In [65]:
features = pd.merge(freq_per_time, sales_type, on = 'custid')

In [66]:
features = pd.merge(features, shop_time, on = 'custid')

In [67]:
features = pd.merge(features, dtdt_mean, on = 'custid')

In [68]:
def floor_transfer_a(x) :
    return  {
        "무역점" : 0,
        "본점" : 1,
        "신촌점" : 0,
        "천호점" : 1
    }.get(x)

def floor_transfer_m(x) :  
    return  {
        "무역점" : 1/10*13,
        "본점" : 1/6*13,
        "신촌점" : 1/11*13,
        "천호점" : 1
    }.get(x)
data['floor'] = (data.str_nm.apply(floor_transfer_a))*data.str_nm.apply(floor_transfer_m).astype(int)

In [69]:
features = pd.merge(features, data.groupby('custid')['floor'].mean().reset_index(), on = 'custid')

In [70]:
features['af_work_rate'] = features['af_work']/(features['af_work'] + features['bf_work'])
features['bf_work_rate'] = features['bf_work']/(features['af_work'] + features['bf_work'])

In [71]:
# 아침구매/ 낮구매/ 밤구매
freq_sales_cat = pd.pivot_table(index='custid',columns='sales_time_cat',values='tot_amt',aggfunc=np.size, fill_value=0, data=data.query("tot_amt>0")[['custid','sales_time_cat','tot_amt']]).reset_index()
features = pd.merge(features, freq_sales_cat,on='custid')

In [72]:
refund_freq=data.query('tot_amt<0').groupby('custid')['sales_day'].count().reset_index()
features = pd.merge(features, refund_freq, left_on = 'custid',right_on='custid',how='left')
features.rename(columns = {'sales_day': 'refund_freq'},inplace=True)
features.fillna(0,inplace=True)

In [73]:
sales_freq=data.query('tot_amt>0').groupby('custid')['sales_day'].count().reset_index()
features = pd.merge(features, sales_freq, left_on = 'custid',right_on='custid',how='left')
features.rename(columns = {'sales_day': 'sales_freq'},inplace=True)

In [74]:
# 구매 빈도 특성 추가
all_freq=data.groupby('custid')['sales_day'].count().reset_index()
features = pd.merge(features, all_freq,  left_on = 'custid',right_on='custid',how='left')
features.rename(columns = {'sales_day': 'all_freq'},inplace=True)

In [75]:
features['refund_sales_rate']=features['refund_freq']/features['sales_freq']
features['refund_all_rate'] = features['refund_freq']/features['all_freq']

방문한 날짜의 갯수

In [76]:
data_aaa=data.groupby(['custid','sales_month','sales_day'])['sales_dayofweek'].count().reset_index()

In [77]:
data_aaa.sales_dayofweek=1

In [78]:
data_aaa=data_aaa.groupby('custid')['sales_dayofweek'].sum().reset_index()

In [79]:
features = pd.merge(features, data_aaa, on = 'custid')

In [80]:
features['sales_freq_mean']=features['sales_freq']/features['sales_dayofweek']

평일에 방문하는 평균 시간

In [81]:
vuddlf=data[data.weekend==0].groupby('custid')['sales_time'].mean().reset_index()

In [82]:
features = pd.merge(features, vuddlf, how='left',on = 'custid')

In [83]:
wkdtjdgus=data[data.weekend==0].groupby('custid')['sales_time'].agg([('sales_time_mod', lambda x: x.value_counts().index[0])]).reset_index()

In [84]:
features = pd.merge(features, wkdtjdgus, how='left',on = 'custid')

주말에 방문하는 평균 시간

In [85]:
wnakf=data[data.weekend==1].groupby('custid')['sales_time'].mean().reset_index()

In [86]:
features = pd.merge(features, wnakf, how='left',on = 'custid')

In [87]:
wkdtjdgus=data[data.weekend==1].groupby('custid')['sales_time'].agg([('sales_time_mod', lambda x: x.value_counts().index[0])]).reset_index()

In [88]:
features = pd.merge(features, wkdtjdgus, how='left',on = 'custid')

In [89]:
# 수입상품 관련 열
sales_import=data.groupby('custid')['import_flg'].agg('sum').reset_index()
features = pd.merge(features, sales_import, on = 'custid')

features.rename(columns = {'import_flg': 'sales_import'},inplace=True)

In [90]:
# 수입상품 구매 빈도 추가
features['import_rate']=features['sales_import']/features['sales_freq']

In [91]:
# 수입상품 환불 빈도 추가
features['import_refund_rate']=features['sales_import']/(features['refund_freq']+1)

In [92]:
# 계절별 구매 빈도 추가
def f1(x):
    if 3 <= x <= 5 :
        return('spring_sales')
    elif 6 <= x <= 8 :
        return('summer_sales')
    elif 9 <= x <= 11 :
        return('fall_sales')
    else :
        return('winter_sales')    

In [93]:
data['season_sales']=data['sales_month'].apply(f1)

In [94]:
season = pd.pivot_table(data, index='custid', columns='season_sales', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()
features = pd.merge(features, season, how='left',on = 'custid')

In [95]:
#계절별 구매만 빈도
season_sales_freq = pd.pivot_table(index='custid',columns='season_sales',values='tot_amt',aggfunc=np.size, fill_value=0, data=data.query("tot_amt>0")[['custid','season_sales','tot_amt']]).reset_index()
features = pd.merge(features, season_sales_freq, how='left',on = 'custid')

In [96]:
#계절별 환불만 빈도
season_refund_freq = pd.pivot_table(index='custid',columns='season_sales',values='tot_amt',aggfunc=np.size, fill_value=0, data=data.query("tot_amt<0")[['custid','season_sales','tot_amt']]).reset_index()
features = pd.merge(features, season_refund_freq, how='left',on = 'custid')
features.fillna(0,inplace=True)

In [97]:
features['sales_fall_rate']=features.fall_sales_x/(features.fall_sales_x+features.spring_sales_x+features.summer_sales_x+features.winter_sales_x)

In [98]:
features['sales_spring_rate']=features.spring_sales_x/(features.fall_sales_x+features.spring_sales_x+features.summer_sales_x+features.winter_sales_x)

In [99]:
features['sales_summer_rate']=features.summer_sales_x/(features.fall_sales_x+features.spring_sales_x+features.summer_sales_x+features.winter_sales_x)

In [100]:
features['sales_winter_rate']=features.winter_sales_x/(features.fall_sales_x+features.spring_sales_x+features.summer_sales_x+features.winter_sales_x)

In [101]:
features['refund_fall_rate']=features.fall_sales_y/(features.fall_sales_y+features.spring_sales_y+features.summer_sales_y+features.winter_sales_y)

In [102]:
features['refund_fall_rate']=features.fall_sales_y/(features.fall_sales_y+features.spring_sales_y+features.summer_sales_y+features.winter_sales_y)

In [103]:
features['refund_spring_rate']=features.spring_sales_y/(features.fall_sales_y+features.spring_sales_y+features.summer_sales_y+features.winter_sales_y)

In [104]:
features['refund_summer_rate']=features.summer_sales_y/(features.fall_sales_y+features.spring_sales_y+features.summer_sales_y+features.winter_sales_y)

In [105]:
features['refund_winter_rate']=features.winter_sales_y/(features.fall_sales_y+features.spring_sales_y+features.summer_sales_y+features.winter_sales_y)

In [106]:
features['all_fall_rate']=features.fall_sales/(features.fall_sales+features.spring_sales+features.summer_sales+features.winter_sales)

In [107]:
features['all_spring_rate']=features.spring_sales/(features.fall_sales+features.spring_sales+features.summer_sales+features.winter_sales)

In [108]:
features['all_summer_rate']=features.summer_sales/(features.fall_sales+features.spring_sales+features.summer_sales+features.winter_sales)

In [109]:
features['all_winter_rate']=features.winter_sales/(features.fall_sales+features.spring_sales+features.summer_sales+features.winter_sales)

In [110]:
# 주말 구매 비율
features['wd_sales_rate'] = (data.groupby('custid')['weekend'].sum().reset_index()['weekend']/features['sales_freq'])*100

월별 구매액의 합, 평균

In [111]:
month_sales_mn=data.groupby(['custid','sales_month'])['net_amt'].mean().unstack().fillna(0)

In [112]:
month_sales_mn.columns = ['mon_sale_mn'+ str(column) for column in month_sales_mn.columns]
month_sales_mn = month_sales_mn.reset_index()

In [113]:
month_sales_sm=data.groupby(['custid','sales_month'])['net_amt'].sum().unstack().fillna(0)

In [114]:
month_sales_sm.columns = ['mon_sale_sm'+ str(column) for column in month_sales_sm.columns]
month_sales_sm = month_sales_sm.reset_index()

In [115]:
features = pd.merge(features, month_sales_sm, how='left',on = 'custid')

In [116]:
features = pd.merge(features, month_sales_mn, how='left',on = 'custid')

In [117]:
# 구매 월 평균
features['month_mean'] = data.groupby('custid')['sales_month'].mean().reset_index()['sales_month']

In [118]:
# 자주 가는 월 - 원래 자주가는 
#f = pd.pivot_table(data, index='custid', columns='sales_month', values='tot_amt', 
#                   aggfunc=np.size, fill_value=0).reset_index()
f=pd.pivot_table(data[data.tot_amt>0], index='custid', columns='sales_month', values='tot_amt', 
                   aggfunc=np.size, fill_value=0).reset_index()

In [119]:
f.columns=['custid', '1_mmff', '2_mmff', '3_mmff', '4_mmff', '5_mmff', '6_mmff', '7_mmff', '8_mmff', '9_mmff', '10_mmff', '11_mmff', '12_mmff']

In [120]:
features = pd.merge(features, f, how='left',on = 'custid')

금액 관리

In [121]:
refund_net_sum=data[data.net_amt<0].groupby('custid')['net_amt'].sum().reset_index()
features = pd.merge(features, refund_net_sum, how='left',on = 'custid')

In [122]:
net_saels_all=data.groupby('custid')['net_amt'].agg([('tot_sales_all','sum'),('mean_amt_all','mean')])
features = pd.merge(features, net_saels_all, how='left',on = 'custid').fillna(0)

In [123]:
net_sales_0_all=data[data.net_amt>0].groupby('custid')['net_amt'].agg([('tot_sales','sum'),('max_sales_amt','max'),('min_sales_amt','min'),('mean_sales_amt','mean')])
features = pd.merge(features, net_sales_0_all, how='left',on = 'custid').fillna(0)

In [124]:
net_refund_all=data[data.net_amt<0].groupby('custid')['net_amt'].agg([('tot_refund','sum'),('max_refund_amt','max'),('min_refund_amt','min'),('mean_refund_amt','mean')])
features = pd.merge(features, net_refund_all, how='left',on = 'custid').fillna(0)

In [125]:
features[['mean_amt_all','mean_sales_amt','mean_refund_amt']] =round(features[['mean_amt_all','mean_sales_amt','mean_refund_amt']],2)

In [126]:
features['tot_sales_freq_mean']=features['tot_sales']/features['sales_dayofweek']

In [127]:
# 총 할인금액
features['tot_dis_amt'] = data.groupby('custid')['dis_amt'].sum().reset_index()['dis_amt']

In [128]:
# 최대, 평균 할인율
features['max_dis'] = data.groupby('custid')['dis_rate'].max().reset_index()['dis_rate']
features['mean_dis'] = data.groupby('custid')['dis_rate'].mean().reset_index()['dis_rate']

In [129]:
# 할인 받은 빈도, 비율
dis_rate=data.query("dis_rate>0").groupby('custid')['dis_rate'].count().reset_index()
features = pd.merge(features, dis_rate, how='left',on='custid').fillna(0)
features.rename(columns = {'dis_rate': 'dis_freq'},inplace=True)

In [130]:
features['dis_freq_rate'] = features['dis_freq']/features['sales_freq']

In [131]:
# 할부 빈도, 비율/ 평균 할부개월
inst_ft = data.query("inst_mon != 1").groupby('custid')['inst_mon'].agg([('inst_freq','count'),('inst_avg','mean'),('inst_max','max')])
features = pd.merge(features, inst_ft, how='left',on = 'custid').fillna(0)
features['inst_freq_rate'] = features['inst_freq']/features['sales_freq']
features['inst_freq_all_rate'] = features['inst_freq']/features['all_freq']

In [132]:
# 할부 총 달
inst_ft_sum = data.groupby('custid')['inst_mon'].agg([('inst_sum','sum')])
features = pd.merge(features, inst_ft_sum, how='left',on = 'custid').fillna(0)

In [133]:
features['tot_inst_amt']=features['tot_sales']/features['inst_sum']

In [134]:
# 할부 총 달
sales_noinst = data.groupby('custid')['inst_fee'].agg([('inst_fee_sum','sum')])
features = pd.merge(features, sales_noinst, how='left',on = 'custid').fillna(0)

In [135]:
#def inst_avg(x):
#    if x >=3:
#        return 1
#    else:
#        return 0
#
#features['inst_quan'] = features.inst_avg.apply(inst_avg)

In [136]:
# 자주가는 지점이 어딘지 원핫인코딩 한 것
data['str_nm'] = data['str_nm'].apply(lambda x: 'mu' if x=='무역점' else('bon' if x=='본점' else('cheon' if x=='천호점' else 'sin')))

In [137]:
f = data.groupby(['custid','str_nm'])['str_nm'].agg([('str_freq','count')]).reset_index()

In [138]:
lst = []
for i in tqdm_notebook(f['custid'].unique()) : 
    f2 = f.query("custid == @i")
    lst.append(f2.sort_values(by='str_freq',ascending=False)['str_nm'].iloc[0])

features['str_freq'] = lst

  0%|          | 0/35967 [00:00<?, ?it/s]

In [139]:
features = pd.get_dummies(features, 'str_freq')

아이디 별 매장별 몇번씩 갔는지

In [140]:
str_freq_all=data.groupby(['custid','str_nm'])['str_nm'].count().unstack().fillna(0)

In [141]:
str_freq_all.columns = ['store_'+ str(column) for column in str_freq_all.columns]
str_freq_all = str_freq_all.reset_index()

In [142]:
features = pd.merge(features, str_freq_all, how='left',on = 'custid')

매장 별로 얼마나 구매를 하였는지 총구매액

In [143]:
str_sale_sum=data[data.net_amt>0].groupby(['custid','str_nm'])['net_amt'].sum().unstack().fillna(0)

In [144]:
str_sale_sum.columns = ['str_sales'+ str(column) for column in str_sale_sum.columns]
str_sale_sum = str_sale_sum.reset_index()

In [145]:
features = pd.merge(features, str_sale_sum, how='left',on = 'custid')

In [146]:
str_refund_sum=data[data.net_amt<0].groupby(['custid','str_nm'])['net_amt'].sum().unstack().fillna(0)

In [147]:
str_refund_sum.columns = ['str_refund'+ str(column) for column in str_refund_sum.columns]
str_refund_sum = str_refund_sum.reset_index()

In [148]:
features = pd.merge(features, str_refund_sum, how='left',on = 'custid')

In [149]:
y['age'] = y['group'].str[1:].astype(int)

In [150]:
y

,custid,group,age
0,0,M30,30
1,2,M30,30
2,3,F20,20
3,4,F20,20
4,5,M40,40
...,...,...,...
21582,29995,M50,50
21583,29996,F20,20
21584,29997,M30,30
21585,29998,F20,20


In [151]:
# 선호 브랜드
df = pd.merge(trn, y, on='custid')

In [152]:
twn_prefer_brd = df.query("age>=20 & age<30").brd_nm.value_counts().index[1:11].tolist()
thr_prefer_brd = df.query("age>=30 & age<40").brd_nm.value_counts().index[1:11].tolist()
for_prefer_brd = df.query("age>=40 & age<50").brd_nm.value_counts().index[1:11].tolist()
fiv_prefer_brd = df.query("age>=50 ").brd_nm.value_counts().index[1:11].tolist()

In [153]:
def map_twn_brd(x):
    if x == twn_prefer_brd[0]:
        x=10
    elif x == twn_prefer_brd[1]:
        x=9
    elif x == twn_prefer_brd[2]:
        x=8
    elif x == twn_prefer_brd[3]:
        x=7
    elif x == twn_prefer_brd[4]:
        x=6
    elif x == twn_prefer_brd[5]:
        x=5
    elif x == twn_prefer_brd[6]:
        x=4
    elif x == twn_prefer_brd[7]:
        x=3
    elif x == twn_prefer_brd[8]:
        x=2
    elif x == twn_prefer_brd[9]:
        x=1
    else:
        x=0
    return x

In [154]:
def map_thr_brd(x):
    if x == thr_prefer_brd[0]:
        x=10
    elif x == thr_prefer_brd[1]:
        x=9
    elif x == thr_prefer_brd[2]:
        x=8
    elif x == thr_prefer_brd[3]:
        x=7
    elif x == thr_prefer_brd[4]:
        x=6
    elif x == thr_prefer_brd[5]:
        x=5
    elif x == thr_prefer_brd[6]:
        x=4
    elif x == thr_prefer_brd[7]:
        x=3
    elif x == thr_prefer_brd[8]:
        x=2
    elif x == thr_prefer_brd[9]:
        x=1
    else:
        x=0
    return x

In [155]:
def map_for_brd(x):
    if x == for_prefer_brd[0]:
        x=10
    elif x == for_prefer_brd[1]:
        x=9
    elif x == for_prefer_brd[2]:
        x=8
    elif x == for_prefer_brd[3]:
        x=7
    elif x == for_prefer_brd[4]:
        x=6
    elif x == for_prefer_brd[5]:
        x=5
    elif x == for_prefer_brd[6]:
        x=4
    elif x == for_prefer_brd[7]:
        x=3
    elif x == for_prefer_brd[8]:
        x=2
    elif x == for_prefer_brd[9]:
        x=1
    else:
        x=0
    return x

In [156]:
def map_fiv_brd(x):
    if x == fiv_prefer_brd[0]:
        x=10
    elif x == fiv_prefer_brd[1]:
        x=9
    elif x == fiv_prefer_brd[2]:
        x=8
    elif x == fiv_prefer_brd[3]:
        x=7
    elif x == fiv_prefer_brd[4]:
        x=6
    elif x == fiv_prefer_brd[5]:
        x=5
    elif x == fiv_prefer_brd[6]:
        x=4
    elif x == fiv_prefer_brd[7]:
        x=3
    elif x == fiv_prefer_brd[8]:
        x=2
    elif x == fiv_prefer_brd[9]:
        x=1
    else:
        x=0
    return x

In [157]:
data['twn_brd_point'] = data['brd_nm'].apply(map_twn_brd)
data['thr_brd_point'] = data['brd_nm'].apply(map_thr_brd)
data['for_brd_point'] = data['brd_nm'].apply(map_for_brd)
data['fiv_brd_point'] = data['brd_nm'].apply(map_fiv_brd)

In [158]:
twn_brd_point=data.groupby('custid')['twn_brd_point'].mean().reset_index()
thr_brd_point=data.groupby('custid')['thr_brd_point'].mean().reset_index()
for_brd_point=data.groupby('custid')['for_brd_point'].mean().reset_index()
fiv_brd_point=data.groupby('custid')['fiv_brd_point'].mean().reset_index()

In [159]:
features = pd.merge(features, twn_brd_point, how='left',on='custid')
features = pd.merge(features, thr_brd_point, how='left',on='custid')
features = pd.merge(features, for_brd_point, how='left',on='custid')
features = pd.merge(features, fiv_brd_point, how='left',on='custid')

In [160]:
data['buyer_nm'] = data['buyer_nm'].str.replace('행사장.*','행사장')
data['buyer_nm'] = data['buyer_nm'].str.replace('피혁A','피혁')
data['buyer_nm'] = data['buyer_nm'].str.replace('피혁B','피혁')

data['buyer_nm'] = data['buyer_nm'].map({'화장품':'a', '수입명품':'b', '캐릭터캐주얼':'c', '트래디셔널캐주얼':'d', '유아동복':'e', '니트단품':'f', '영캐주얼':'g',
       '엘레강스캐주얼':'h', '가전':'i', '섬유':'j', '장신구':'k', '조리욕실':'l', '스포츠':'m', '침구수예':'n', '피혁':'o', '일반식품':'p',
       '유니캐주얼':'q', '정장셔츠':'r', '디자이너부띠끄':'s', '문화완구':'t', '타운모피':'u', '조리식품':'v', '기타바이어':'w',
       '도자기크리스탈':'x', '가구':'y', '생활용품':'z', '행사장':'aa', '청과곡물':'bb', '점외':'cc'})

buyer = pd.pivot_table(index='custid',columns='buyer_nm',values='tot_amt',aggfunc=np.size,fill_value=0,data=data.query("tot_amt>0")[['custid','buyer_nm','tot_amt']]).reset_index()
features = pd.merge(features, buyer, on='custid')

In [161]:
#구매상품군 다양성
n = data.part_nm.nunique()
f = data.groupby('custid')['part_nm'].agg([('part_unique', lambda x: len(x.unique()) / n)]).reset_index()
features = pd.merge(features, f, on='custid')

In [162]:
#브랜드 다양성
n=data.brd_nm.nunique()
f = data.groupby('custid')['brd_nm'].agg([('brd_unique', lambda x: len(x.unique()) / n)]).reset_index()
features = pd.merge(features, f, on='custid')

In [163]:
#매장 이용 다양성
n = 4
f = data.groupby('custid')['str_nm'].agg([('str_unique', lambda x: len(x.unique()) / n)]).reset_index()
features = pd.merge(features, f, on='custid')

In [164]:
#위에서는 출근 시간 9~17시 퇴근시간 이후로 했는데 좀더 세분화
def f2(x):
    if 9 <= x < 12 :
        return('9_12_sale')
    elif 12 <= x < 14 :
        return('12_14_sale')
    elif 14 <= x < 16 :
        return('14_16_sale')
    elif 16 <= x < 18 :
        return('16_18_sale')
    else :
        return('18_sale')  

data['timeslot2'] = data.sales_time.apply(f2)

In [165]:
new_sales_type = pd.pivot_table(index='custid',columns='timeslot2',values='tot_amt',aggfunc=np.size, fill_value=0,data = data.query('tot_amt>0')[['custid','timeslot2','tot_amt']]).reset_index()

In [166]:
features=pd.merge(features, new_sales_type, on='custid')

In [167]:
features['12_14_sale_ratio']=features['12_14_sale']/(features['12_14_sale'] + features['14_16_sale']+features['16_18_sale']+features['18_sale']+features['9_12_sale'])
features['14_16_sale_ratio']=features['14_16_sale']/(features['12_14_sale'] + features['14_16_sale']+features['16_18_sale']+features['18_sale']+features['9_12_sale'])
features['16_18_sale_ratio']=features['16_18_sale']/(features['12_14_sale'] + features['14_16_sale']+features['16_18_sale']+features['18_sale']+features['9_12_sale'])
features['18_sale_ratio']=features['18_sale']/(features['12_14_sale'] + features['14_16_sale']+features['16_18_sale']+features['18_sale']+features['9_12_sale'])
features['9_12_sale_ratio']=features['9_12_sale']/(features['12_14_sale'] + features['14_16_sale']+features['16_18_sale']+features['18_sale']+features['9_12_sale'])

In [168]:
#충동지수 충동구매를 하는사람들은 누구일까? 여유가있는 나이가 많은 사람들이 충동구매를 할거같다. 곱하기대신 더하기도 생각해보기

features['chungdong_sale']=(features.max_sales_amt/features.tot_sales)
features['chungdong_sale_freq']= features.chungdong_sale*features.sales_freq

In [169]:
features['mean_min_sales']=features.mean_sales_amt+features.min_sales_amt

In [170]:
a=features[['custid','mean_min_sales']]

In [171]:
df_LEFT_JOIN = pd.merge(data, a, left_on='custid', right_on='custid', how='left')

In [172]:
df_LEFT_JOIN['chungdong']=df_LEFT_JOIN.net_amt >df_LEFT_JOIN.mean_min_sales

In [173]:
chung=df_LEFT_JOIN.groupby('custid')['chungdong'].sum().reset_index()

In [174]:
features = pd.merge(features, chung, left_on='custid', right_on='custid', how='left')

In [175]:
a=data.groupby(['custid','sales_month','sales_day','sales_dayofweek'])['custid'].size().unstack().reset_index()

In [176]:
a.iloc[:,3:]=a.iloc[:,3:]/a.iloc[:,3:]

In [177]:
a=a.groupby('custid')[['금','목','수','월','일','토','화']].sum()

In [178]:
features = pd.merge(features, a, on='custid')

In [179]:
features = features.rename(columns={'월':'mon_opn','화':'tue_opn','수':'wen_opn','목':'thr_opn','금':'fri_opn','토':'sat_opn','일':'sun_opn'})

In [180]:
features['mon_opn_rate']=features.mon_opn/features.sales_dayofweek
features['tue_opn_rate']=features.tue_opn/features.sales_dayofweek
features['wen_opn_rate']=features.wen_opn/features.sales_dayofweek
features['thr_opn_rate']=features.thr_opn/features.sales_dayofweek
features['fri_opn_rate']=features.fri_opn/features.sales_dayofweek
features['sat_opn_rate']=features.sat_opn/features.sales_dayofweek
features['sun_opn_rate']=features.sun_opn/features.sales_dayofweek

In [181]:
a=data[data.net_amt>0].groupby(['custid','sales_month','sales_day','sales_dayofweek'])['net_amt'].sum().unstack().reset_index()

In [182]:
a=a.groupby('custid')[['금','목','수','월','일','토','화']].sum()

In [183]:
features = pd.merge(features, a, on='custid')

In [184]:
features = features.rename(columns={'월':'mon_pri','화':'tue_pri','수':'wen_pri','목':'thr_pri','금':'fri_pri','토':'sat_pri','일':'sun_pri'})

In [185]:
features['mon_pri_rate']=features.mon_pri/features.tot_sales
features['tue_pri_rate']=features.tue_pri/features.tot_sales
features['wen_pri_rate']=features.wen_pri/features.tot_sales
features['thr_pri_rate']=features.thr_pri/features.tot_sales
features['fri_pri_rate']=features.fri_pri/features.tot_sales
features['sat_pri_rate']=features.sat_pri/features.tot_sales
features['sun_pri_rate']=features.sun_pri/features.tot_sales

충동구매를 하고 환불을 한 횟수

In [186]:
features['max_min_sales']=features['max_sales_amt']-features['min_sales_amt']

In [187]:
def shopfnc(m):
    if m < 5:
        return 1
    elif m < 12.94 :
        return 2 
    elif m < 21.538 :
        return 3 
    elif m < 33.909 :
        return 4
    else:
        return 5

In [188]:
features['shop_time_mean']= features.shop_time.apply(lambda x : shopfnc(x))

In [189]:
for i in np.arange(0.1,1,0.1):
    print(features.tot_sales.quantile(q=i))

249406.0
498646.0000000001
795650.0
1165370.0000000002
1622598.0
2254104.0
3107333.5999999996
4541631.800000001
7485838.200000001


In [190]:
# 10분위별로 (1 ~ 10 범위)
def fnc(m) : 
    if m < 249406.0:
        return 1
    elif m <498646 :
        return 2 
    elif m < 795650.0 :
        return 3 
    elif m < 1165370:
        return 4 
    elif m < 1622598.0 :
        return 5 
    elif m < 2254104 :
        return 6 
    elif m < 3107333:
        return 7 
    elif m < 4541631 :
        return 8
    elif m < 7485838 :
        return 9 
    else:
        return 10

features['tot_sales_amt_quan']= features.tot_sales.apply(lambda x : fnc(x))

In [191]:
for i in np.arange(0.1,1,0.1):
    print(features.max_sales_amt.quantile(q=i))

98400.0
149000.0
199000.0
261250.0
330600.0
416000.0
534850.0
736250.0
1180000.0


In [192]:
# 10분위별로 가격선호도 조사 (1 ~ 10 범위)
def fnc2(m) : 
    if m < 98400.0 :
        return 1
    elif m < 149000.0 :
        return 2 
    elif m < 199000.0 :
        return 3 
    elif m < 261250.0 :
        return 4 
    elif m < 330600.0 :
        return 5 
    elif m < 416000.0 :
        return 6 
    elif m < 534850.0 :
        return 7 
    elif m < 736250.0 :
        return 8
    elif m < 1180000.0 :
        return 9 
    else:
        return 10

features['max_amt_quan']= features.max_sales_amt.apply(lambda x : fnc2(x))

In [193]:
#아동상품 구매건수
f = data[data.tot_amt > 0].groupby('custid')['part_nm'].agg([('baby_sales', lambda x: list(x).count('아동')+list(x).count('케주얼,구두,아동')+list(x).count('아동문화')+list(x).count('아동,스포츠'))]).reset_index()
features = pd.merge(features, f, on='custid')

In [194]:
features['baby_sales_freq']=features['baby_sales']/features.sales_freq

In [195]:
#할부 총합
sales_installment=data.groupby('custid')['inst_mon'].agg('sum').reset_index()
features = pd.merge(features, sales_installment, on='custid')

In [196]:
data['team_nm'] = data['team_nm'].map({'잡화가용팀':'aa', '의류패션팀':'bb', '식품팀':'cb', '인터넷백화점':'dd'})

In [197]:
f = data.groupby(['custid', 'team_nm'])['custid'].count().unstack()
f = f.fillna(0)
f.columns = ['Team_'+ str(column) for column in f.columns]
f = f.reset_index()

In [198]:
features = pd.merge(features, f, on = 'custid')

In [199]:
features['food_rate']=features.Team_cb/(features.Team_aa+features.Team_bb+features.Team_cb+features.Team_dd)

In [200]:
features['cloth_rate']=features.Team_bb/(features.Team_aa+features.Team_bb+features.Team_cb+features.Team_dd)

In [201]:
features['stuff_rate']=features.Team_aa/(features.Team_aa+features.Team_bb+features.Team_cb+features.Team_dd)

In [202]:
food_list=['조리식품','축산물','농산물','수산물']
t1 = data[data.net_amt>0].query("pc_nm == @food_list").groupby('custid')['net_amt'].agg([('food_sales', np.sum)])
features= pd.merge(features,t1, on = 'custid', how='left').fillna(0)
features['food_sales_rate'] = features['food_sales'] / features['tot_sales']

In [203]:
st=f"(net_amt>0) and (pc_nm=='화장품')"

In [204]:
t2 = data.query(st).groupby('custid')['net_amt'].agg([('face_goods_sales', np.sum)])
features= pd.merge(features,t2, on = 'custid', how='left').fillna(0)
features['face_sales_rate'] = features['face_goods_sales'] / features['tot_sales']

In [205]:
t2 = data.query(st).groupby('custid')['net_amt'].agg([('face_goods_freq', np.size)])
features= pd.merge(features,t2, on = 'custid', how='left').fillna(0)
features['face_goods_rate'] = features['face_goods_freq'] / features['sales_freq']

In [206]:
data['pc_nm'] = data['pc_nm'].str.replace('디자이너부띠끄','디자이너부띠크')
data['pc_nm'] = data['pc_nm'].str.replace('디자이너부틱','디자이너부띠크')
data['pc_nm'] = data['pc_nm'].str.replace('트래디셔널','트레디셔널')
data['pc_nm'] = data['pc_nm'].str.replace('침구/수예','침구수예')
data['pc_nm'] = data['pc_nm'].str.replace('침구,수예','침구수예')
data['pc_nm'] = data['pc_nm'].str.replace('피혁A','피혁')
data['pc_nm'] = data['pc_nm'].str.replace('피혁B','피혁')
data['pc_nm'] = data['pc_nm'].str.replace('캐릭터케쥬얼','캐릭터캐쥬얼')
data['pc_nm'] = data['pc_nm'].str.replace('캐릭터캐주얼','캐릭터캐쥬얼')
data['pc_nm'] = data['pc_nm'].str.replace('진캐주얼','진캐쥬얼')
data['pc_nm'] = data['pc_nm'].str.replace('진케주얼','진캐쥬얼')
data['pc_nm'] = data['pc_nm'].str.replace('로얄부틱','로얄부띠크')
data['pc_nm'] = data['pc_nm'].str.replace('로얄부띠끄','로얄부띠크')

In [207]:
twn_trend_brd = df.query("age>=20 & age<30").pc_nm.value_counts().index[1:11].tolist()
thr_trend_brd = df.query("age>=30 & age<40").pc_nm.value_counts().index[1:11].tolist()
for_trend_brd = df.query("age>=40 & age<50").pc_nm.value_counts().index[1:11].tolist()
fiv_trend_brd = df.query("age>=50").pc_nm.value_counts().index[1:11].tolist()


def map_twn_pc(x):
    if x == twn_trend_brd[0]:
        x=10
    elif x == twn_trend_brd[1]:
        x=9
    elif x == twn_trend_brd[2]:
        x=8
    elif x == twn_trend_brd[3]:
        x=7
    elif x == twn_trend_brd[4]:
        x=6
    elif x == twn_trend_brd[5]:
        x=5
    elif x == twn_trend_brd[6]:
        x=4
    elif x == twn_trend_brd[7]:
        x=3
    elif x == twn_trend_brd[8]:
        x=2
    elif x == twn_trend_brd[9]:
        x=1
    else:
        x=0
    return x

def map_thr_pc(x):
    if x == thr_trend_brd[0]:
        x=10
    elif x == thr_trend_brd[1]:
        x=9
    elif x == thr_trend_brd[2]:
        x=8
    elif x == thr_trend_brd[3]:
        x=7
    elif x == thr_trend_brd[4]:
        x=6
    elif x == thr_trend_brd[5]:
        x=5
    elif x == thr_trend_brd[6]:
        x=4
    elif x == thr_trend_brd[7]:
        x=3
    elif x == thr_trend_brd[8]:
        x=2
    elif x == thr_trend_brd[9]:
        x=1
    else:
        x=0
    return x

def map_for_pc(x):
    if x == for_trend_brd[0]:
        x=10
    elif x == for_trend_brd[1]:
        x=9
    elif x == for_trend_brd[2]:
        x=8
    elif x == for_trend_brd[3]:
        x=7
    elif x == for_trend_brd[4]:
        x=6
    elif x == for_trend_brd[5]:
        x=5
    elif x == for_trend_brd[6]:
        x=4
    elif x == for_trend_brd[7]:
        x=3
    elif x == for_trend_brd[8]:
        x=2
    elif x == for_trend_brd[9]:
        x=1
    else:
        x=0
    return x

def map_fiv_pc(x):
    if x == fiv_trend_brd[0]:
        x=10
    elif x == fiv_trend_brd[1]:
        x=9
    elif x == fiv_trend_brd[2]:
        x=8
    elif x == fiv_trend_brd[3]:
        x=7
    elif x == fiv_trend_brd[4]:
        x=6
    elif x == fiv_trend_brd[5]:
        x=5
    elif x == fiv_trend_brd[6]:
        x=4
    elif x == fiv_trend_brd[7]:
        x=3
    elif x == fiv_trend_brd[8]:
        x=2
    elif x == fiv_trend_brd[9]:
        x=1
    else:
        x=0
    return x


data['twn_pc_point'] = data['pc_nm'].apply(map_twn_pc)
data['thr_pc_point'] = data['pc_nm'].apply(map_thr_pc)
data['for_pc_point'] = data['pc_nm'].apply(map_for_pc)
data['fiv_pc_point'] = data['pc_nm'].apply(map_fiv_pc)


twn_pcnm_point=data.groupby('custid')['twn_pc_point'].mean().reset_index()
thr_pcnm_point=data.groupby('custid')['thr_pc_point'].mean().reset_index()
for_pcnm_point=data.groupby('custid')['for_pc_point'].mean().reset_index()
fiv_pcnm_point=data.groupby('custid')['fiv_pc_point'].mean().reset_index()


features = pd.merge(features, twn_pcnm_point, how='left',on='custid')
features = pd.merge(features, thr_pcnm_point, how='left',on='custid')
features = pd.merge(features, for_pcnm_point, how='left',on='custid')
features = pd.merge(features, fiv_pcnm_point, how='left',on='custid')


In [208]:
f = data[data.tot_amt > 0].groupby('custid')['pc_nm'].agg([('young_pc', lambda x: list(x).count('캐릭터정장')+list(x).count('영트랜디')+list(x).count('영캐릭터캐쥬얼')+list(x).count('트랜디 케쥬얼')+list(x).count('캐릭터캐쥬얼')+list(x).count('장신구')+list(x).count('피혁')+list(x).count('섬유')+list(x).count('소품')+list(x).count('핸드백')+list(x).count('(주)현스포츠아쌤'))]).reset_index()
features = pd.merge(features, f, on='custid')

In [209]:
features['20goods_rate']=features['young_pc']/features.sales_freq

In [210]:
#50대선호

f = data[data.tot_amt > 0].groupby('custid')['pc_nm'].agg([('butique_freq', lambda x: list(x).count('로얄부띠크')+list(x).count('디자이너부띠크'))]).reset_index()
features = pd.merge(features, f, on='custid')

In [211]:
features['50goods_rate']=features['butique_freq']/features.sales_freq

In [212]:
#트랜드 명품 구매비율
f = data[data.tot_amt > 0].groupby('custid')['pc_nm'].agg([('expensive_freq', lambda x: list(x).count('명품토탈')+list(x).count('명품시즌'))]).reset_index()
features = pd.merge(features, f, on='custid')

In [213]:
features['20~30expensive_rate']=features['expensive_freq']/features.sales_freq

In [214]:
data['pc_nm'] = data['pc_nm'].map({'화장품':'face_goods', '명품토탈':'expensive_total', '캐릭터정장':'chr_suit',  '수예':'suae', '유아/완구':'baby_wangu', '패션슈즈':'fashoin_shoes',
       '가전':'electronic', '영트랜디':'young_trend', '조리식품':'instant_food', '스포츠':'sports', '아동':'baby', '트래디셔널캐쥬얼':'trady_ca', '가전/문화':'ele_cultur', '셔츠':'shirts',
       '란제리':'rangeri', '식기':'food_case', '영캐릭터캐쥬얼':'young_chr', '골프':'golf', '캐릭터캐쥬얼':'chr_cha', '엘레강스캐쥬얼':'elegangs', '트레디셔널':'tradi',
       '정장':'suit', '일반식품':'normal_food', '아동복':'baby_cloth', '축산물':'pig', '미확인pc':'unknown_pc','농산물':'vegetable','트랜디 케쥬얼':'trendi cha',
        '영트렌디':'young_trendi','여성캐주얼':'girl_cha','문화용품':'culture_goods','생활용품':'lsang','문화':'culture','니트,단품,모피':'neat_mofi','섬유잡화':'sumyou_goods','내의란제리':'inrangeri',
        '진캐쥬얼':'jean_ca3','여성구두':'girl_shoes','명품시즌':'expensive_season','남여구두':'man_girl','장신구':'jangsingu','수산물':'susanmul','악세사리':'accesory','섬유':'sumyou',
        '어덜트캐주얼':'adult_cha','소품':'sopeum','캐주얼':'cha','미씨캐주얼':'messi_cha','영캐주얼':'young_cha','타운란제리':'townrangeri',
        '핸드백':'handbag','수입명품':'suip_goods','디자이너부띠크':'designier_beatuty','쇼핑보증':'shooping_bo','미씨캐릭터':'messi_cha5',
        '가구':'gagu','캐쥬얼':'cha2','니트/단품':'neat_one','남성잡화':'man_jobs','(주)현스포츠아쌤':'sportsam','침구수예':'chimgu4','패션시즌':'fashion_season',
        '수입의류':'suip_cloth','용기보증':'youngi','사이버쇼핑':'cyber_shopping'})

In [215]:
#주구매상품
f = data.groupby('custid')['pc_nm'].agg([
    ('main_goods', lambda x: x.value_counts().index[0])
]).reset_index()

In [216]:
features = features.merge(f, how='left')
features = pd.get_dummies(features, 'main_goods')

In [217]:
f_p = data[data.tot_amt>0].groupby('pc_nm')['tot_amt'].agg([
    ('main_goods_price', 'mean')
]).reset_index()

In [218]:
f = pd.merge(f, f_p,  left_on = 'main_goods',right_on='pc_nm',how='left')

In [219]:
features = pd.merge(features, f[['custid','main_goods_price']],  left_on = 'custid',right_on='custid',how='left')

In [220]:
pc_count=data[data.tot_amt>0].groupby(['custid','pc_nm'])['pc_nm'].count().unstack().fillna(0)

pc_count.columns = ['pc_size_'+ str(column) for column in pc_count.columns]
pc_count = pc_count.reset_index()

features = pd.merge(features, pc_count, how='left',on = 'custid')

In [221]:

data['part_nm'] = data['part_nm'].map({'명품잡화':'aaa', '잡화파트':'bbb', '남성의류':'ccc', '골프/유니캐쥬얼':'ddd', '가정용품':'eee', '아동':'fff', '잡화':'ggg',
       '영어덜트캐쥬얼':'hhh', '영라이브':'iii', '공산품':'jjj', '스포츠캐쥬얼':'kkk', '여성캐주얼':'lll', '여성정장':'mmm', '케주얼,구두,아동':'nnn', '남성정장스포츠':'ooo', '아동문화':'ppp',
       '로얄부띠끄':'qqq', '공산품파트':'rrr', '생식품파트':'sss', '여성의류파트':'ttt', '가정용품파트':'uuu', '아동,스포츠':'vvv', '패션잡화':'www',
       '영캐릭터':'abbb', '영플라자':'abbc', '생식품':'abbd', '스포츠캐주얼':'abbe', '여성캐쥬얼':'abbf', '로얄부틱':'abbg','인터넷백화점':'abbh'})


In [222]:
#주구매상품분류
f = data.groupby('custid')['part_nm'].agg([
    ('main_part', lambda x: x.value_counts().index[0])
]).reset_index()

In [223]:
features = features.merge(f, how='left')
features = pd.get_dummies(features, 'main_part')

In [224]:
f_p = data[data.tot_amt>0].groupby('part_nm')['tot_amt'].agg([
    ('main_part_price', 'mean')
]).reset_index()

In [225]:
f = pd.merge(f, f_p,  left_on = 'main_part',right_on='part_nm',how='left')

In [226]:
features = pd.merge(features, f[['custid','main_part_price']],  left_on = 'custid',right_on='custid',how='left')

In [227]:
part_count=data[data.tot_amt>0].groupby(['custid','part_nm'])['part_nm'].count().unstack().fillna(0)

part_count.columns = ['part_size_'+ str(column) for column in part_count.columns]
part_count = part_count.reset_index()

features = pd.merge(features, part_count, how='left',on = 'custid')

In [228]:
#내점 당 구매건수 / 내점당 구매액
features['one_sales_freq']=features.sales_freq/features.sales_dayofweek
features['one_tot_amt']=features.tot_sales/features.sales_dayofweek

In [229]:
data['part_pc'] = data['part_nm'].astype(str) + "_" + data['pc_nm'].astype(str)

In [230]:
#주구매상품분류
f = data.groupby('custid')['part_pc'].agg([
    ('main_part_pc', lambda x: x.mode()[0])
]).reset_index()

In [231]:
features = features.merge(f, how='left')
features = pd.get_dummies(features, 'main_part_pc')

In [232]:
f_p = data[data.tot_amt>0].groupby('part_pc')['tot_amt'].agg([
    ('main_part_pc_price', 'mean')
]).reset_index()

In [233]:
f = pd.merge(f, f_p,  left_on = 'main_part_pc',right_on='part_pc',how='left')

In [234]:
features = pd.merge(features, f[['custid','main_part_pc_price']],  left_on = 'custid',right_on='custid',how='left')

In [235]:
part_count=data[data.tot_amt>0].groupby(['custid','part_pc'])['part_pc'].count().unstack().fillna(0)

part_count.columns = ['part_pc_size_'+ str(column) for column in part_count.columns]
part_count = part_count.reset_index()

features = pd.merge(features, part_count, how='left',on = 'custid')

In [236]:
features = features.loc[:,~features.T.duplicated()]; features

,custid,1_stf,9_stf,10_stf,11_stf,12_stf,13_stf,14_stf,15_stf,16_stf,...,part_pc_size_www_face_goods,part_pc_size_www_fashion_season,part_pc_size_www_girl_shoes,part_pc_size_www_handbag,part_pc_size_www_jangsingu,part_pc_size_www_nan,part_pc_size_www_rangeri,part_pc_size_www_shooping_bo,part_pc_size_www_sumyou,part_pc_size_www_unknown_pc
0,0,0,0,0,0,2,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0,0,1,1,0,1,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0,0,0,2,3,2,3,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,0,0,0,0,0,0,4,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0,0,0,8,1,2,0,1,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,49988,0,0,0,0,3,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35963,49990,0,0,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35964,49992,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35965,49993,0,0,0,0,0,0,0,0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [237]:
features1=features.copy()

In [238]:
i_max=[]
for i in list(features.columns):
    if features[i].max()>50000:
        i_max.append(i)

In [239]:
i_max_min=[]
for i in i_max:
    if features[i].min()>0:
        i_max_min.append(i)

In [240]:
features[i_max_min]=np.log1p(features[i_max_min])

In [241]:
# 데이터 분할
X_train = pd.DataFrame({'custid':trn['custid'].unique()})
X_train = pd.merge(X_train, features, how='left',on='custid')

y_train = y['age']

X_test = pd.DataFrame({'custid':tst['custid'].unique()})
X_test = pd.merge(X_test, features, how='left',on='custid')

In [242]:
X_train.drop('custid',axis=1,inplace=True)
X_test.drop('custid',axis=1,inplace=True)

In [243]:
X_train.shape, X_test.shape, y_train.shape

((21587, 721), (14380, 721), (21587,))

In [244]:
X_columns=X_train.columns

In [247]:
X = pd.concat([X_train,X_test])

In [249]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

#X_train_scaled = scaler.fit_transform(X_train)
#X_test_scaled = scaler.transform(X_test)
X_all = scaler.fit_transform(X)

In [252]:
ALL = pd.DataFrame(X_all,columns=X_columns);ALL

,1_stf,9_stf,10_stf,11_stf,12_stf,13_stf,14_stf,15_stf,16_stf,17_stf,...,part_pc_size_www_face_goods,part_pc_size_www_fashion_season,part_pc_size_www_girl_shoes,part_pc_size_www_handbag,part_pc_size_www_jangsingu,part_pc_size_www_nan,part_pc_size_www_rangeri,part_pc_size_www_shooping_bo,part_pc_size_www_sumyou,part_pc_size_www_unknown_pc
0,-0.005273,-0.088843,-0.284814,-0.411768,-0.089984,-0.620230,-0.651158,-0.678094,-0.696307,-0.697887,...,-0.404966,-0.149921,-0.173264,-0.149937,-0.19268,-0.318958,-0.158148,-0.005273,-0.285985,-0.017491
1,-0.005273,-0.088843,0.398751,-0.187084,-0.528885,-0.389632,-0.435693,-0.678094,-0.503130,-0.498945,...,-0.404966,-0.149921,-0.173264,-0.149937,-0.19268,-0.318958,-0.158148,-0.005273,-0.285985,-0.017491
2,-0.005273,-0.088843,-0.284814,0.037600,0.129466,-0.159034,-0.004763,-0.678094,-0.696307,0.296823,...,-0.404966,-0.149921,-0.173264,-0.149937,-0.19268,-0.318958,-0.158148,-0.005273,-0.285985,-0.017491
3,-0.005273,-0.088843,-0.284814,-0.411768,-0.528885,-0.620230,0.210703,-0.678094,-0.696307,-0.697887,...,-0.404966,-0.149921,-0.173264,-0.149937,-0.19268,-0.318958,-0.158148,-0.005273,-0.285985,-0.017491
4,-0.005273,-0.088843,-0.284814,1.385706,-0.309434,-0.159034,-0.651158,-0.481193,-0.116775,0.296823,...,-0.404966,-0.149921,-0.173264,-0.149937,-0.19268,-0.318958,-0.158148,-0.005273,-0.285985,-0.017491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35962,-0.005273,-0.088843,-0.284814,-0.411768,0.129466,-0.620230,-0.651158,-0.678094,-0.696307,-0.697887,...,-0.404966,-0.149921,-0.173264,-0.149937,-0.19268,-0.318958,-0.158148,-0.005273,-0.285985,-0.017491
35963,-0.005273,-0.088843,-0.284814,-0.411768,-0.528885,-0.620230,-0.651158,-0.481193,-0.696307,-0.697887,...,-0.404966,-0.149921,-0.173264,-0.149937,-0.19268,-0.318958,-0.158148,-0.005273,-0.285985,-0.017491
35964,-0.005273,-0.088843,-0.284814,-0.411768,-0.528885,-0.620230,-0.651158,-0.678094,-0.696307,-0.498945,...,-0.404966,-0.149921,-0.173264,-0.149937,-0.19268,-0.318958,-0.158148,-0.005273,-0.285985,-0.017491
35965,-0.005273,-0.088843,-0.284814,-0.411768,-0.528885,-0.620230,-0.651158,-0.678094,-0.503130,-0.300003,...,0.059019,-0.149921,-0.173264,-0.149937,-0.19268,-0.318958,-0.158148,-0.005273,-0.285985,-0.017491


In [260]:
T = pd.Series(IDtrain)
TE = pd.Series(IDtest)

In [277]:
X_train_scaled = ALL[:21587]
X_test_scaled = ALL[21587:].reset_index().iloc[:,1:]

In [284]:
#X_train_scaled=pd.DataFrame(X_train_scaled,columns=X_columns)
#X_test_scaled=pd.DataFrame(X_test_scaled,columns=X_columns)

In [281]:
X_train_scaled = pd.concat([X_train_scaled, train_mean_vector], axis=1)
X_test_scaled = pd.concat([X_test_scaled, test_mean_vector], axis=1)

In [283]:
X_train_scaled = pd.concat([X_train_scaled, train_mean_vector_2], axis=1)
X_test_scaled = pd.concat([X_test_scaled, test_mean_vector_2], axis=1)

In [284]:
X_train_scaled.shape, X_test_scaled.shape, y_train.shape

((21587, 921), (14380, 921), (21587,))

In [285]:
X_train_scaled.to_csv('2.2nd_feature_0612.csv' , index = False)

In [286]:
X_test_scaled.to_csv('2.2nd_feature_te_0612.csv' , index = False)

In [287]:
X_train_scaled

,1_stf,9_stf,10_stf,11_stf,12_stf,13_stf,14_stf,15_stf,16_stf,17_stf,...,90,91,92,93,94,95,96,97,98,99
0,-0.005273,-0.088843,-0.284814,-0.411768,-0.089984,-0.620230,-0.651158,-0.678094,-0.696307,-0.697887,...,-0.165226,0.226791,0.023077,0.197462,0.159934,0.134166,-0.084717,0.058027,0.233149,0.146704
1,-0.005273,-0.088843,0.398751,-0.187084,-0.528885,-0.389632,-0.435693,-0.678094,-0.503130,-0.498945,...,-0.212440,0.137763,-0.042228,0.076557,0.097435,0.117673,-0.081022,0.041562,0.235905,0.100596
2,-0.005273,-0.088843,-0.284814,0.037600,0.129466,-0.159034,-0.004763,-0.678094,-0.696307,0.296823,...,-0.185209,0.162313,-0.024075,0.080504,0.102624,0.079979,-0.165694,0.105364,0.314384,0.118440
3,-0.005273,-0.088843,-0.284814,-0.411768,-0.528885,-0.620230,0.210703,-0.678094,-0.696307,-0.697887,...,-0.051455,0.020150,-0.073568,0.343153,0.258331,0.246874,-0.016653,0.158458,0.133428,0.105987
4,-0.005273,-0.088843,-0.284814,1.385706,-0.309434,-0.159034,-0.651158,-0.481193,-0.116775,0.296823,...,-0.179383,0.158299,-0.097633,0.092266,0.095512,0.130884,-0.039362,0.080870,0.225688,0.169993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21582,-0.005273,-0.088843,1.082317,-0.187084,1.226718,1.224554,1.934424,-0.087390,1.235466,2.087299,...,-0.087380,0.180822,-0.091415,0.093610,0.085881,0.134376,-0.136256,0.114941,0.086217,0.114949
21583,-0.005273,-0.088843,-0.284814,-0.411768,0.348917,0.302162,-0.651158,-0.678094,-0.116775,-0.300003,...,-0.095967,0.147775,-0.108850,0.159752,0.103106,0.166150,-0.158565,0.209097,0.172259,0.031050
21584,-0.005273,-0.088843,-0.284814,-0.411768,-0.309434,0.071564,0.210703,0.700217,-0.116775,-0.300003,...,-0.083197,0.235958,-0.067154,0.069837,0.179463,0.112109,-0.186623,0.223913,0.138100,0.103012
21585,-0.005273,-0.088843,-0.284814,-0.411768,-0.528885,0.532760,-0.220228,-0.284291,-0.309953,-0.697887,...,-0.041586,0.228185,-0.060137,0.210461,0.191180,0.128997,-0.136482,0.187943,0.338590,-0.023360
